In [ ]:
# !pip install flash-attn --no-build-isolation

In [1]:
import json
import outlines
import torch
from typing import List, Optional
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from datasets import load_dataset,Dataset
from rich import print
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import Dict,Tuple,Any
from outlines import Generator
from outlines.types import JsonSchema
from outlines import from_transformers
import os
import tqdm
# os.environ["TRANSFORMERS_VERBOSITY"] = "info"
load_dotenv()

True

In [2]:
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

In [3]:
PARA = 0.5
MODEL_NAME = f"Qwen/Qwen2.5-{PARA}B-Instruct"
MODEL_PATH = "./Models"
DATASET_NAME = "Vishva007/RBI-Circular-QA-Dataset"
SEED = 42
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") 
GOOGLE_MODEL_ID="gemini-2.0-flash"
GOOGLE_TEMPERATURE=0.01
GOOGLE_MAX_TOKENS=256
EVAL_SET_SIZE = 10
MERGED_MODEL_OUTPUT_DIR = f"./Models/Qwen2.5-{PARA}B-Instruct-RBI-QA-Merged"
MODEL_FINETUNED_REPO_ID = f"Vishva007/Qwen2.5-{PARA}B-Instruct-RBI-QA-Finetuned"

In [4]:
print(f"Using Model: {MODEL_NAME} with {PARA}B parameters")

Using Model: Qwen/Qwen2.5-0.5B-Instruct with 0.5B parameters

In [5]:
import os
print(f"Current working directory: {os.getcwd()}")
absolute_model_path = os.path.abspath(MODEL_PATH)
print(f"Absolute model path: {absolute_model_path}")


Current working directory: /workspaces/Fine_Tune_Qwen/Code/eval-LLM

Absolute model path: /workspaces/Fine_Tune_Qwen/Code/eval-LLM/Models

In [6]:
print(f"Loading dataset: {DATASET_NAME}")
print(f"Using seed: {SEED}")

Loading dataset: Vishva007/RBI-Circular-QA-Dataset

Using seed: 42

In [7]:
try:
    dataset = load_dataset(DATASET_NAME,split="eval")
    print("Dataset loaded successfully!")
    print(dataset) # Print the dataset structure to see available splits
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Please ensure the dataset name is correct and you have an active internet connection.")


Dataset loaded successfully!

Dataset({
    features: ['document', 'filename', 'model_name', 'regulation_area', 'applicable_to', 'issued_on', 'key_topics',
'chunks_text', 'is_table', 'question', 'answer', 'evaluation_criteria', 'category', 'estimated_difficulty', 
'rephrased_question', 'rephrased_answer'],
    num_rows: 100
})

In [8]:
number_of_examples_for_eval = EVAL_SET_SIZE

# Get the total number of examples in full_dataset
num_examples = len(dataset)

# Calculate the start index for the last EVAL_SET_SIZE examples
start_index = num_examples - number_of_examples_for_eval

# Create a list of indices for the last 1EVAL_SET_SIZE0 examples
indices_to_select = list(range(start_index, num_examples))

# Use the .select() method to create a new Dataset object containing only those indices
eval_dataset = dataset.select(indices_to_select)


In [9]:
print("dataset type:", type(dataset))
print("eval_dataset type:", type(dataset))

dataset type: <class 'datasets.arrow_dataset.Dataset'>

eval_dataset type: <class 'datasets.arrow_dataset.Dataset'>

In [10]:
class ResponseFormat(BaseModel):
    answer : str

class BatchResponseFormat(BaseModel):
    responses: List[ResponseFormat] = Field(..., description="List of responses for each question in the batch.")   
    
class EvaluationResult(BaseModel):
    score: int = Field(..., description="Score 1 if the answer fully satisfies ALL the specified criteria, 0 otherwise.", ge=0, le=1)


In [11]:
if torch.cuda.is_available():
    print("GPU is Used!")
else:
    print("CPU is Used!")

GPU is Used!

In [12]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True,
    cache_dir=MODEL_PATH,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=MODEL_PATH)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


In [ ]:
import sys


def generate_responses_from_prompts(model, tokenizer, prompts: list[str]) -> list[str]:
    """
    Generates responses for a batch of prompts using a loaded causal language model and tokenizer.

    Args:
        model: The pre-trained causal language model.
        tokenizer: The tokenizer corresponding to the model.
        prompts (list[str]): A list of user prompts for which to generate responses.

    Returns:
        list[str]: A list of generated responses, corresponding to each input prompt.
    """
    if model is None or tokenizer is None:
        print("Model or tokenizer not loaded. Cannot generate responses.")
        return ["Error: Model not loaded." for _ in prompts]

    sys_prompt = """
        You are a highly knowledgeable AI assistant with expertise in Indian banking and financial regulations, 
        particularly those outlined in Reserve Bank of India (RBI) circulars.
        Your task is to answer questions based on the RBI circulars and related financial regulations.
        - Return only the answer to the question.
        - Do not include any additional information or explanations.
    """
    # Prepare messages for each prompt in the batch
    batched_messages = []
    for prompt in prompts:
        batched_messages.append([
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": prompt}
        ])

    # Apply chat template to each set of messages and collect texts
    batched_texts = [
        tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        ) for messages in batched_messages
    ]

    print(f"Processing {len(prompts)} prompts in batch...")

    try:
        # Tokenize the entire batch
        # padding=True pads sequences to the longest sequence in the batch
        # return_tensors="pt" returns PyTorch tensors
        model_inputs = tokenizer(batched_texts, return_tensors="pt", padding=True).to(model.device)

        # Generate responses for the entire batch
        # max_new_tokens controls the maximum number of tokens to generate per response
        # **model_inputs unpacks the dictionary into keyword arguments
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
            temperature=0.01,
            top_p=0.95,
            top_k=50
        )

        # Calculate the length of the input IDs for each original prompt
        # This is necessary to slice out only the newly generated tokens
        input_lengths = model_inputs.input_ids.shape[1]

        # Slice generated_ids to get only the new tokens for each item in the batch
        # We iterate through each generated sequence and remove the input part
        batch_generated_ids_only = [
            output_ids[input_lengths:] for output_ids in generated_ids
        ]

        # Decode the generated tokens back to text for the entire batch
        responses = tokenizer.batch_decode(batch_generated_ids_only, skip_special_tokens=True)
        print("Batch processing complete.")
        return responses

    except Exception as e:
        print(f"Error during generation: {e}")
        return ["Error generating response." for _ in prompts] # Return error for each prompt



In [15]:
prompt_to_process = eval_dataset[:5]["question"]
print(f"Processing prompt: {prompt_to_process}")

Processing prompt: ['What is the general role of the Banking Regulation Act of 1949 in the context of banking 
companies in India?', 'How should Nodal/Focal Point branches report April 2023 transactions, and how should they 
differentiate them from March Residual Transactions in their reporting?', "What is the procedure for regulated 
entities to handle requests for delisting from the Security Council's ISIL (Da'esh) and Al-Qaida Sanctions List, 
and what alternative options are available for individuals or entities seeking removal from the list?", 'How should
Credit Information Companies provide individuals with access to their free annual credit report?', 'How can card 
issuing banks enable Card-on-File (CoFT) token generation, and what are the key requirements for customer consent 
and token availability?']

In [16]:
batch_responses = generate_responses_from_prompts(model, tokenizer, prompt_to_process)

# Print the generated responses
print("\n--- Generated Responses ---")
for i, response in enumerate(batch_responses):
    print(f"Prompt {i+1}: \"{prompt_to_process[i]}\"")
    print(f"Response {i+1}:\n{response}\n{'-'*30}\n")

Processing 5 prompts in batch...

Batch processing complete.

--- Generated Responses ---

Prompt 1: "What is the general role of the Banking Regulation Act of 1949 in the context of banking companies in 
India?"

Response 1:
The Banking Regulation Act of 1949 was primarily designed to regulate the banking industry in India. It aimed to 
ensure fair competition among banks, protect consumers, and maintain public trust in the banking system. The act 
established various regulatory bodies such as the Reserve Bank of India (RBI), which oversaw the implementation of 
the act, and the Central Bank of India (CBI), which provided guidance and oversight. This act also introduced 
several new rules and guidelines for bank operations, including minimum capital requirements, lending limits, and 
deposit insurance.
------------------------------

...

Response 3:
To handle requests for delisting from the Security Council's ISIL (Da'esh) and Al-Qaida Sanctions List, regulated 
entities should follow these steps:

1. **Review the Request**: The entity must review the request to ensure it complies with the relevant regulations 
and guidelines set by the International Monetary Fund (IMF), the World Bank, and other international organizations.

2. **Submit the Request**: Once the entity has reviewed the request, they should submit it to the appropriate 
regulatory body or authority within their jurisdiction.

3. **Follow Up**: After submission, the entity should follow up with the regulatory body to confirm receipt and 
provide any necessary documentation or information.

For individuals or entities seeking removal from the list, there are several alternatives available:

- **International Law**: Individuals can seek legal advice to understand the implications of being removed from the
list under international law, including potential sanctions or penalties.
  
- **Alternative Sanctions**: Some countries have implemented measures to remove individuals from the list through 
diplomatic channels or other means that do not involve direct removal from the list itself.

- **Criminal Proceedings**: In some cases, individuals may be subject to criminal proceedings if they are found to 
have committed serious crimes against humanity or war crimes during their time in the list.

It's important for regulated entities to carefully consider all options before making decisions regarding delisting
or removal from the list.
------------------------------

Prompt 4: "How should Credit Information Companies provide individuals with access to their free annual credit 
report?"

Response 4:
Credit Information Companies should provide individuals with access to their free annual credit report by following
these steps:

1. **Obtain Consent**: Obtain written consent from the individual before providing them with their credit report.

2. **Provide Access**: Provide the individual with an opportunity to review their credit report at their 
convenience.

3. **Maintain Privacy**: Ensure that the data provided is kept confidential and used solely for personal purposes.

4. **Comply with Regulations**: Adhere to all applicable laws and regulations regarding privacy and data 
protection.

5. **Security Measures**: Implement robust security measures to protect the confidentiality and integrity of the 
data being accessed.

6. **Training**: Provide training to employees who handle credit reports to ensure they understand how to use this 
information responsibly.

7. **Reporting Requirements**: Follow reporting requirements set by the relevant regulatory bodies such as the Data
Protection Act 1988 in India.

By following these guidelines, Credit Information Companies can provide individuals with accurate and secure access
to their credit reports while respecting privacy and compliance with regulations.
------------------------------

Prompt 5: "How can card issuing banks enable Card-on-File (CoFT) token generation, and what are the key 
requirements for customer consent and token availability?"

Response 5:
Card Issuing Banks can enable CoFT token generation through various methods such as API integration, mobile apps, 
or web services. To ensure customer consent and token availability, banks must comply with regulatory requirements 
like the Payment Services Act 2018, which mandates that customers have explicit consent before using their bank 
cards.

Key requirements for customer consent:

1. **Consent from Customers**: Banks need to obtain explicit consent from customers before they use their bank 
cards for transactions. This typically involves providing clear instructions about how the bank will handle the 
transaction and obtaining written confirmation of the customer's agreement.

2. **Privacy Policy**: Banks should clearly state in their privacy policy that they collect and use personal data 
for CoFT purposes. They must also provide an option for customers to opt-out of this service if they do not want 
their data used for CoFT.

3. **Data Security Measures**: Banks must implement robust security measures to protect the data collected by CoFT 
tokens. This includes encryption, secure storage, and regular audits to ensure compliance with data protection 
laws.

4. **Transparency**: Banks must be transparent about how CoFT tokens work and how they are used. This includes 
disclosing the purpose of the tokens, the process for generating them, and the benefits provided to customers.

5. **Documentation**: Banks must maintain detailed records of all transactions involving CoFT tokens, including 
details of the customer's identity, the type of transaction, and the date and time of the transaction.

6. **Training and Awareness**: Banks should provide training to their staff on CoFT token usage and the importance 
of customer consent. Staff should understand the legal implications of using CoFT tokens and how to manage customer
expectations regarding these tokens.

By adhering to these requirements, banks can effectively manage CoFT tokens while ensuring customer consent and 
maintaining trust in their services.
------------------------------

In [17]:
# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
        model=GOOGLE_MODEL_ID, 
        temperature=GOOGLE_TEMPERATURE,
        max_tokens=GOOGLE_MAX_TOKENS,
        timeout=None,
        api_key=GOOGLE_API_KEY,
    )

In [21]:


def generate_answers_with_transformers(
    model,
    eval_dataset: Dataset,
    batch_size: int = 5,
) -> List[Dict[str, str]]:
    """
    Generates answers for a dataset using an transformers LLM.

    Args:
        model: The loaded HuggingFace model for transformers.
        eval_dataset (datasets.Dataset): The dataset containing 'question' fields.
        batch_size (int): The number of samples to process in each batch.
        device (str): The device to run the model on (e.g., "cuda" or "cpu").

    Returns:
        List[Dict[str, str]]: A list of dictionaries, each containing the original question
                              and the 'generated_answer' from Outlines.
    """

    generated_data = []

    print(f"Generating answers in batches of {batch_size}...")
    for i in range(0, len(eval_dataset), batch_size):
        batch_slice = eval_dataset.select(list(range(i, min(i + batch_size, len(eval_dataset)))))
        
        inputs = [item['question'] for item in batch_slice.to_list()]

        try:
            
            batch_response_instance = generate_responses_from_prompts(model, tokenizer, inputs)
            
            for original_item, transformers_output in zip(batch_slice.to_list(), batch_response_instance):
                generated_data.append({
                    "question": original_item['question'],
                    "generated_answer": transformers_output
                })
        except Exception as e:
            print(f"Error during transformers generation for batch starting at index {i}: {e}")
            for original_item in batch_slice.to_list():
                 generated_data.append({
                    "question": original_item['question'],
                    "generated_answer": "\\ (Generation Failed)"
                })

    print("transformers answer generation complete.")
    return generated_data

In [22]:
generated_answers = generate_answers_with_transformers(
    model,
    eval_dataset,
    batch_size=5,
)

Generating answers in batches of 5...

Processing 5 prompts in batch...

Batch processing complete.

Processing 5 prompts in batch...

Batch processing complete.

transformers answer generation complete.

In [23]:
print(generated_answers)

[
    {
        'question': 'What is the general role of the Banking Regulation Act of 1949 in the context of banking 
companies in India?',
        'generated_answer': 'The Banking Regulation Act of 1949 was primarily designed to regulate the banking 
industry in India. It aimed to ensure fair competition among banks, protect consumers, and maintain public trust in
the banking system. The act established various regulatory bodies such as the Reserve Bank of India (RBI), which 
oversaw the implementation of the act, and the Central Bank of India (CBI), which provided guidance and oversight. 
This act also introduced several new rules and guidelines for bank operations, including minimum capital 
requirements, lending limits, and deposit insurance.'
    },
    {
        'question': 'How should Nodal/Focal Point branches report April 2023 transactions, and how should they 
differentiate them from March Residual Transactions in their reporting?',
        'generated_answer': "To report Nodal/Focal Point branches for April 2023 transactions, you should:\n\n1. 
Identify all transactions that occurred during April 2023.\n2. Determine which of these transactions were part of 
the Nodal/Focal Point branch's operations.\n3. Report these transactions as if they were March residual 
transactions.\n\nFor example:\n- If a transaction was made on April 15th, it would be reported as an April 2023 
transaction.\n- If a transaction was made on April 20th, it would be considered a March residual transaction 
because it did not occur during April 2023.\n\nThis approach ensures that both types of transactions are accurately
categorized and reported according to the rules set by the RBI."
    },
    {
        'question': "What is the procedure for regulated entities to handle requests for delisting from the 
Security Council's ISIL (Da'esh) and Al-Qaida Sanctions List, and what alternative options are available for 
individuals or entities seeking removal from the list?",
        'generated_answer': "To handle requests for delisting from the Security Council's ISIL (Da'esh) and 
Al-Qaida Sanctions List, regulated entities should follow these steps:\n\n1. **Review the Request**: The entity 
must review the request to ensure it complies with the rules set by the International Monetary Fund (IMF), the 
World Bank, and other international organizations.\n\n2. **Submit the Request**: Once the entity has reviewed the 
request, they should submit it to the appropriate regulatory body or authority within their jurisdiction.\n\n3. 
**Follow Up**: After submission, the entity should follow up with the regulatory body to confirm receipt and 
provide any necessary documentation or information.\n\nFor individuals or entities seeking removal from the list, 
there are several alternatives available:\n\n- **International Criminal Court (ICC)**: Individuals can file a 
complaint against individuals or entities involved in terrorist activities, including those linked to Da'esh and 
Al-Qaida, at the ICC. This process involves filing a complaint with the ICC and following its procedures.\n\n- 
**United Nations Security Council**: If an individual or entity is found guilty of terrorism-related offenses, they
may be subject to sanctions under the UN Security Council. This includes being removed from the list of sanctioned 
entities.\n\n- **Alternative Sanctions**: In some cases, individuals or entities may face alternative sanctions 
such as economic penalties, travel bans, or restrictions on their ability to conduct business 
internationally.\n\nIt's important for regulated entities to understand the specific requirements and procedures 
for each type of sanction and to consult with legal counsel if needed."
    },
    {
        'question': 'How should Credit Information Companies provide individuals with access to their free annual 
credit report?',
        'generated_answer': "Credit Information Companies should provide individuals with access to their free 
annual c

In [24]:
def evaluate_answers_with_gemini(
    eval_dataset: Dataset,
    generated_answers_data: List[Dict[str, str]],
    llm: ChatGoogleGenerativeAI,
) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
    """
    Evaluates generated answers using a Gemini LLM, returning only the score in the structured output.

    Args:
        eval_dataset (datasets.Dataset): The original dataset containing 'question',
                                          'answer' (ground truth), and 'evaluation_criteria' fields.
        generated_answers_data (List[Dict[str, str]]): A list of dictionaries, where each
                                                       dictionary contains the original question
                                                       and the 'generated_answer'.
        llm (ChatGoogleGenerativeAI): The LangChain Gemini LLM instance.

    Returns:
        tuple: A tuple containing:
               - list: A list of dictionaries, where each dictionary contains the original question,
                       ground truth answer, generated answer, and evaluation score.
               - dict: A summary dictionary of the evaluation.
    """

    SYSTEM_MESSAGE = """
    You are an expert evaluator tasked with determining if an answer satisfies specified evaluation criteria.

    You will receive:
    1. A question
    2. The evaluation criteria
    3. The model's answer to evaluate

    Provide your score based on the criteria:
    - Score 1 if the answer fully satisfies ALL the specified criteria
    - Score 0 if it fails to meet ANY of the criteria

    Format your response as a JSON object with a single key 'score' containing just the number 1 or 0.
    For example: {{"score": 1}}
    """.strip()

    PROMPT_TEMPLATE = """
    Question: {question}
    Evaluation Criteria: {evaluation_criteria}
    Model Answer: {model_answer}

    Provide the evaluation score in the specified JSON format.
    """.strip()

    gemini_eval_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", SYSTEM_MESSAGE),
            ("human", PROMPT_TEMPLATE),
        ]
    )

    gemini_eval_chain = gemini_eval_prompt | llm.with_structured_output(EvaluationResult)

    results = []
    total_evaluated = 0
    total_score = 0
    failed_evaluations_due_to_error = 0

    print(f"Evaluating answers with Gemini...")
    combined_data = list(zip(eval_dataset.to_list(), generated_answers_data))

    for original_item, generated_item in combined_data:
        total_evaluated += 1

        question = original_item['question']
        ground_truth_answer = original_item['answer']
        evaluation_criteria = original_item['evaluation_criteria']
        generated_answer = generated_item['generated_answer']

        gemini_input = {
            "question": question,
            "evaluation_criteria": evaluation_criteria,
            "model_answer": generated_answer,
        }

        score_val = 0

        try:
            structured_result = gemini_eval_chain.invoke(gemini_input)
            score_val = structured_result.score
            total_score += score_val

        except Exception as e:
            print(f"Error during Gemini evaluation for question: '{question}'. Error: {e}")
            score_val = 0
            failed_evaluations_due_to_error += 1

        results.append({
            "question": question,
            "ground_truth_answer": ground_truth_answer,
            "generated_answer": generated_answer,
            "evaluation_score": score_val,
        })

    # Calculate summary statistics
    successful_eval_count = total_evaluated - failed_evaluations_due_to_error
    percentage_passed_criteria = (total_score / successful_eval_count) * 100 if successful_eval_count > 0 else 0

    summary = {
        "total_evaluations_attempted": total_evaluated,
        "successful_evaluations": successful_eval_count,
        "failed_evaluations_due_to_error": failed_evaluations_due_to_error,
        "total_passed_criteria": total_score,
        "percentage_passed_criteria": percentage_passed_criteria,
        "overall_evaluation_summary": (
            f"Out of {total_evaluated} attempted evaluations, "
            f"{successful_eval_count} were successfully processed by Gemini. "
            f"{failed_evaluations_due_to_error} evaluations encountered an error. "
            f"Among the successfully evaluated answers, {total_score} met all criteria, "
            f"resulting in a {percentage_passed_criteria:.2f}% pass rate."
        )
    }
    print("Gemini evaluation complete.")
    return results, summary


In [25]:
evaluation_results, evaluation_summary = evaluate_answers_with_gemini(
    eval_dataset=eval_dataset, 
    generated_answers_data=generated_answers, 
    llm=llm,
)

Evaluating answers with Gemini...

Gemini evaluation complete.

In [26]:
print(evaluation_results[:10])

[
    {
        'question': 'What is the general role of the Banking Regulation Act of 1949 in the context of banking 
companies in India?',
        'ground_truth_answer': 'The Banking Regulation Act of 1949 governs the functioning of banking companies in 
India, setting standards for their operations, management, and financial stability.',
        'generated_answer': 'The Banking Regulation Act of 1949 was primarily designed to regulate the banking 
industry in India. It aimed to ensure fair competition among banks, protect consumers, and maintain public trust in
the banking system. The act established various regulatory bodies such as the Reserve Bank of India (RBI), which 
oversaw the implementation of the act, and the Central Bank of India (CBI), which provided guidance and oversight. 
This act also introduced several new rules and guidelines for bank operations, including minimum capital 
requirements, lending limits, and deposit insurance.',
        'evaluation_score': 0
    },
    {
        'question': 'How should Nodal/Focal Point branches report April 2023 transactions, and how should they 
differentiate them from March Residual Transactions in their reporting?',
        'ground_truth_answer': 'Nodal/Focal Point branches must submit a monthly statement for April transactions 
to Zonal Accounts Officers/Pay and Accounts Officers in the usual format. To differentiate between April 
transactions (financial year 2023-24) and March Residual Transactions, the statement pertaining to March Residual 
Transactions should be clearly marked as "March Residual Account".',
        'generated_answer': "To report Nodal/Focal Point branches for April 2023 transactions, you should:\n\n1. 
Identify all transactions that occurred during April 2023.\n2. Determine which of these transactions were part of 
the Nodal/Focal Point branch's operations.\n3. Report these transactions as if they were March residual 
transactions.\n\nFor example:\n- If a transaction was made on April 15th, it would be reported as an April 2023 
transaction.\n- If a transaction was made on April 20th, it would be considered a March residual transaction 
because it did not occur during April 2023.\n\nThis approach ensures that both types of transactions are accurately
categorized and reported according to the rules set by the RBI.",
        'evaluation_score': 0
    },
    {
        'question': "What is the procedure for regulated entities to handle requests for delisting from the 
Security Council's ISIL (Da'esh) and Al-Qaida Sanctions List, and what alternative options are available for 
individuals or entities seeking removal from the list?",
        'ground_truth_answer': "Regulated entities should forward any delisting requests they receive to the Joint 
Secretary (CTCR) at the Ministry of Home Affairs for consideration. Individuals or entities seeking removal from 
the Security Council's ISIL (Da'esh) and Al-Qaida Sanctions List can submit their request to an independent 
Ombudsperson appointed by the United Nations Secretary-General.",
        'generated_answer': "To handle requests for delisting from the Security Council's ISIL (Da'esh) and 
Al-Qaida Sanctions List, regulated entities should follow these steps:\n\n1. **Review the Request**: The entity 
must review the request to ensure it complies with the rules set by the International Monetary Fund (IMF), the 
World Bank, and other international organizations.\n\n2. **Submit the Request**: Once the entity has reviewed the 
request, they should submit it to the appropriate regulatory body or authority within their jurisdiction.\n\n3. 
**Follow Up**: After submission, the entity should follow up with the regulatory body to confirm receipt and 
provide any necessary documentation or information.\n\nFor individuals or entities seeking removal from the list, 
there are several alternatives available:\n\n- **International Criminal Court (ICC)**: Individuals can file a 
complaint against individuals or entities i

In [27]:

print("\n--- Evaluation Summary ---")
summary_stats = evaluation_summary
print(f"Total Evaluations Attempted: {summary_stats['total_evaluations_attempted']}")
print(f"Evaluations Failed Due to Error: {summary_stats['failed_evaluations_due_to_error']}")
print(f"Total Answers Passed Criteria (Score 1): {summary_stats['total_passed_criteria']}")
print(f"Percentage of Successfully Evaluated Answers Passing Criteria: {summary_stats['percentage_passed_criteria']:.2f}%")
print(f"Overall Summary: {summary_stats['overall_evaluation_summary']}")

--- Evaluation Summary ---

Total Evaluations Attempted: 10

Evaluations Failed Due to Error: 0

Total Answers Passed Criteria (Score 1): 2

Percentage of Successfully Evaluated Answers Passing Criteria: 20.00%

Overall Summary: Out of 10 attempted evaluations, 10 were successfully processed by Gemini. 0 evaluations 
encountered an error. Among the successfully evaluated answers, 2 met all criteria, resulting in a 20.00% pass 
rate.

In [28]:
import os
import json
def save_evaluation_to_json(evaluation_results: list[dict], evaluation_summary: dict, output_dir: str, filename: str = "gemini_evaluation_report.json"):
    """
    Saves evaluation results and summary to a JSON file in the specified directory.

    Args:
        evaluation_results (list[dict]): A list of dictionaries, where each dictionary
                                         represents the evaluation of a single question-answer pair.
        evaluation_summary (dict): A dictionary containing the overall summary of the evaluation.
        output_dir (str): The directory where the JSON file should be saved.
                          This directory will be created if it does not exist.
        filename (str, optional): The name of the JSON file. Defaults to "gemini_evaluation_report.json".
    """
    # Combine evaluation results and summary into a single dictionary
    output_data = {
        "evaluation_results": evaluation_results,
        "evaluation_summary": evaluation_summary
    }

    # Ensure the output directory exists
    try:
        os.makedirs(output_dir, exist_ok=True)
    except OSError as e:
        print(f"Error creating directory {output_dir}: {e}")
        return

    # Construct the full path for the JSON file
    file_path = os.path.join(output_dir, filename)

    # Write the data to a JSON file
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, indent=4)
        print(f"Successfully wrote evaluation results and summary to {file_path}")
    except IOError as e:
        print(f"Error writing to file {file_path}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [29]:
save_evaluation_to_json(
    evaluation_results=evaluation_results,
    evaluation_summary=evaluation_summary,
    output_dir="./Evaluation_Results",
    filename="qwen2.5-0.5B.json"
)

Successfully wrote evaluation results and summary to ./Evaluation_Results/qwen2.5-0.5B.json